# Set up


## Imports

In [1]:
%matplotlib inline

In [2]:
import sys
import os
from pathlib import Path
import math

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk", font_scale=1, rc=None)
sns.set_style('whitegrid')

In [4]:
# Load the "autoreload" extension
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np

from collections import OrderedDict

from munch import Munch, munchify
from tqdm import tqdm

In [134]:
import pendulum as lum
pdm = lum

In [6]:
import engarde.decorators as ed
import pyparsing as p

In [7]:
# import crunchers.sklearn_helpers.exploration as expl
# import crunchers.statsmodels_helpers.lazy_stats as stats
# import crunchers.pandas_helpers.transformations as xform

In [8]:
import biorep_etl.data.hreg_load_recode as loading
import biorep_etl.data.field_definitions.hreg_redcap_dump as rcd
from biorep_etl.data.parsers import BranchLogicParser

## Constants

### Get info for converting/moving this file to `reports`

In [9]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "IPYNB_NAME = " + "'"+thename+"'";
# kernel.execute(command);

In [10]:
# rv = Munch()

# rv.rep_id = 'yyy-mm-dd'
# rv.desc = 'blank'
# rv.ipnb = Path('{IPYNB_NAME}.ipynb'.format(IPYNB_NAME=IPYNB_NAME))
# rv.html = '{ipnb}.html'.format(ipnb=rv.ipnb.stem)
# rv.files = '{ipnb}_files'.format(ipnb=rv.ipnb.stem)
# rv.reprt_dir = '../../reports/{IPYNB_NAME}/'.format(IPYNB_NAME=IPYNB_NAME)

### Other Constants

## Paths

In [11]:
data_ = "../data/raw/registry/HarvardInflammatoryB_DATA_2017-01-20_0924.csv"
data_dict_ = "../data/raw/registry/HarvardInflammatoryBowelDiseas_DataDictionary_2017-01-20.csv"

## Functions

## Loading

### Data Dict

In [14]:
cb = loading.load_data_dict(data_dict_=data_dict_).dropna(axis=1, how='all')
cb.head(2)

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
subid,registration_visit,NaN,text,Subject ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitdate,registration_visit,Form RV: Registration Visit,text,Date of visit at which patient was consented:,NaN,NaN,date_mdy,NaN,NaN,NaN,NaN,y,NaN,NaN


In [15]:
cb['Field Type'].value_counts()

radio          214
text            80
checkbox        49
dropdown        34
yesno            6
descriptive      5
calc             3
Name: Field Type, dtype: int64

In [16]:
cb[cb['Field Type'] == 'radio'].head()

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
noconsent,registration_visit,NaN,radio,"If no, provide reason","1, Not interested in Research Studies | 2, Pri...",NaN,NaN,NaN,NaN,NaN,"[consent] = ""0""",NaN,NaN,NaN
gender,registration_visit,NaN,radio,Gender,"0, Female | 1, Male | 99, Undefined",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
race,registration_visit,NaN,radio,Race,"1, White or Caucasian | 2, Black or African Am...",Put other for anything not listed here (Middle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitcategory,baseline_and_follow_up,NaN,radio,What type of visit is this?,"1, Baseline Visit | 2, Follow Up Visit",Please make sure this matches the event name a...,NaN,NaN,NaN,NaN,[registration_visit_arm_1][ibddiag] = '1' and ...,y,NaN,NaN
visittype,baseline_and_follow_up,NaN,radio,BV4. Visit Type,"1, Baseline Visit, within 3 months of initial ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='1',NaN,NaN,NaN


In [24]:
cb[cb['Field Type'] == 'dropdown'].head()

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
paperfill,baseline_and_follow_up,NaN,dropdown,Who filled out the paper CRF?**,"0, N/A | 1, Alex | 2, Ben | 3, Emily | 4, Jill...",NaN,NaN,NaN,NaN,NaN,[registration_visit_arm_1][ibddiag] = '1',NaN,NaN,NaN
entertoredcap,baseline_and_follow_up,NaN,dropdown,Who entered the CRF directly into RedCap?**,"6, Alex | 12, Anastasia | 7, Julia | 13, Kate ...",NaN,NaN,NaN,NaN,NaN,[registration_visit_arm_1][ibddiag] = '1',NaN,NaN,NaN
review,baseline_and_follow_up,NaN,dropdown,Who has reviewed this?**,"1, Nobody | 2, An IBD Fellow | 3, Athos | 4, S...",NaN,NaN,NaN,NaN,NaN,[registration_visit_arm_1][ibddiag] = '1',NaN,NaN,NaN
meddraae1,baseline_and_follow_up,Adverse Event or Intolerance to Medication,dropdown,"Mesalamine (Asacol, Canasa, Pentasa, Rowasa, A...","1, 10000496 Acne | 2, 10002199 Anaphylactic sh...",NaN,NaN,NaN,NaN,NaN,"([ibdmed]=""1"" or [ibdmednew] = '1') and [mesa...",NaN,NaN,NaN
meddraae2,baseline_and_follow_up,NaN,dropdown,Sulfasalazine (Azulfidine),"1, 10000496 Acne | 2, 10002199 Anaphylactic sh...",NaN,NaN,NaN,NaN,NaN,"([ibdmed]=""1"" or [ibdmednew] = '1') and [sulfa...",NaN,NaN,NaN


In [161]:
cb['Text Validation Type OR Show Slider Number'].unique()

array([nan, 'date_mdy', 'integer', 'number', 'number_1dp', 'date_dmy'], dtype=object)

In [160]:
dt_vals = {'date_mdy', 'date_dmy'}
list(cb[cb['Text Validation Type OR Show Slider Number'].isin(dt_vals)].index.values)

array(['visitdate', 'dob', 'visitdate1', 'dateentered', 'ibddiag1',
       'onsetdt', 'wtdiagdt', 'htdiagdt', 'prevwtdiagdt', 'prevhtdiagdt',
       'pcdaidoa', 'hbaidoa', 'pucaidoa', 'lab_reference', 'date_labs',
       'dateofarrival', 'surg_a', 'surg_b', 'surg_c', 'surg_d', 'surg_e',
       'surg_f', 'surg_g', 'surg_h', 'surg_i', 'surg_j', 'surg_k',
       'surg_l', 'hosp_a', 'hosp_b', 'hosp_c', 'hosp_d', 'hosp_e',
       'hosp_f', 'hosp_g', 'hosp_h', 'hosp_i', 'hosp_j', 'hosp_k',
       'hosp_l', 'hosp_m'], dtype=object)

### Data

In [171]:
d = loading.load_redcap_dump(data_=data_, data_dict=cb)
d.head()

visitdate consent noconsent  \
subid redcap_event_name                                                      
1     baseline_visit_arm_1                           NaT     NaN       NaN   
      followup_visit_1_arm_1                         NaT     NaN       NaN   
      followup_visit_2_arm_1                         NaT     NaN       NaN   
      registration_visit_arm_1 2011-05-24 00:00:00+00:00     1.0       NaN   
10    baseline_visit_arm_1                           NaT     NaN       NaN   

                               noconsentoth veoibd otherstudies  \
subid redcap_event_name                                           
1     baseline_visit_arm_1              NaN    NaN          NaN   
      followup_visit_1_arm_1            NaN    NaN          NaN   
      followup_visit_2_arm_1            NaN    NaN          NaN   
      registration_visit_arm_1          NaN    NaN          1.0   
10    baseline_visit_arm_1              NaN    NaN          NaN   

                               prior_protocol_number___5  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       1.0   
10    baseline_visit_arm_1                           NaN   

                               prior_protocol_number___1  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       0.0   
10    baseline_visit_arm_1                           NaN   

                               prior_protocol_number___2  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       0.0   
10    baseline_visit_arm_1                           NaN   

                               prior_protocol_number___3  \
subid redcap_event_name                                    
1     baseline_visit_arm_1                           NaN   
      followup_visit_1_arm_1                         NaN   
      followup_visit_2_arm_1                         NaN   
      registration_visit_arm_1                       0.0   
10    baseline_visit_arm_1                           NaN   

                                          ...            hosp_k hosptype_k  \
subid redcap_event_name                   ...                                
1     baseline_visit_arm_1                ...               NaN        NaN   
      followup_visit_1_arm_1              ...               NaN        NaN   
      followup_visit_2_arm_1              ...               NaN        NaN   
      registration_visit_arm_1            ...               NaN        NaN   
10    baseline_visit_arm_1                ...               NaN        NaN   

                               hospibdtype_k hosp_l hosptype_l hospibdtype_l  \
subid redcap_event_name                                                        
1     baseline_visit_arm_1               NaN    NaN        NaN           NaN   
      followup_visit_1_arm_1             NaN    NaN        NaN           NaN   
      followup_visit_2_arm_1             NaN    NaN        NaN           NaN   
      registration_visit_arm_1           NaN    NaN        NaN           NaN   
10    baseline_visit_arm_1               NaN    NaN        NaN           NaN   

                               hosp_m hosptype_m hospibdtype_m  \
subid redcap_event_name                                          
1     baseline_visit_arm_1        NaN        NaN           NaN   
      followup_visit_1_arm_1      NaN   

In [50]:
idx = pd.IndexSlice

In [52]:
np.datetime64conda 

str

# infer_dtypes

In [57]:
type(lum.parse('2011-05-24'))

pendulum.pendulum.Pendulum

In [154]:
try:
    crude_dtypes = loading.infer_crude_dtypes(data_dict=cb, data_path=data_)
except KeyError as exc:
    print(exc.args)

In [156]:
crude_dtypes.redcap_dtypes.abcessess

'radio'

In [151]:
df = loading.load_redcap_dump(data_=data_, data_dict=cb)
col = 'visitdate'

In [169]:
loading.cast_column_as_date(df=df,col='redcap_event_name')
df.visitdate

KeyError: 'redcap_event_name'

In [150]:
df[col].apply(loading.cast_func_ignore_nulls, f=lum.parse)

subid         redcap_event_name       
1             baseline_visit_arm_1                              NaN
              followup_visit_1_arm_1                            NaN
              followup_visit_2_arm_1                            NaN
              registration_visit_arm_1    2011-05-24T00:00:00+00:00
10            baseline_visit_arm_1                              NaN
              followup_visit_1_arm_1                            NaN
              followup_visit_2_arm_1                            NaN
              hospitalizations_arm_1                            NaN
              registration_visit_arm_1    2011-07-05T00:00:00+00:00
100           baseline_visit_arm_1                              NaN
              followup_visit_1_arm_1                            NaN
              hospitalizations_arm_1                            NaN
              registration_visit_arm_1    2011-10-03T00:00:00+00:00
              surgeries_arm_1                                   NaN
1004     

In [80]:
lum.parse(np.NAN)

TypeError: expected string or bytes-like object